In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords

import re

from sklearn.model_selection import train_test_split

import gensim

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM, Embedding
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
data = pd.read_csv("Resources/CL_data/train_K.tsv", sep="\t")

In [4]:
data.head(20)

,id,tweet,label
0,264183816548130816,Gas by my house hit $3.39!!!! I'm going to Cha...,1
1,264249301910310912,Iranian general says Israel's Iron Dome can't ...,-1
2,264105751826538497,with J Davlar 11th. Main rivals are team Polan...,1
3,264094586689953794,"Talking about ACT's && SAT's, deciding where I...",-1
4,254941790757601280,"They may have a SuperBowl in Dallas, but Dalla...",-1
5,264169034155696130,Im bringing the monster load of candy tomorrow...,0
6,263192091700654080,"Apple software, retail chiefs out in overhaul:...",0
7,263398998675693568,@oluoch @victor_otti @kunjand I just watched i...,1
8,260200142420992000,#Livewire Nadal confirmed for Mexican Open in ...,0
9,264087629237202944,@MsSheLahY I didnt want to just pop up... but ...,1


In [5]:
data.shape

(7589, 3)

In [6]:
data["label"].value_counts()

 0    3691
 1    2834
-1    1064
Name: label, dtype: int64

## Cleaning the data

In [7]:
# dropping id column

data = data.drop("id", axis=1)

In [8]:
stopwords = stopwords.words('english')

In [9]:
def tweet_cleaner(tweet):
    tweet = re.sub(r"@\w*", " ", str(tweet).lower()).strip() #removing username
    tweet = re.sub(r'https?://[A-Za-z0-9./]+', " ", str(tweet).lower()).strip() #removing links
    tweet = re.sub(r'[^a-zA-Z]', " ", str(tweet).lower()).strip() #removing sp_char
    tw = []
    
    for text in tweet.split():
        if text not in stopwords:
            tw.append(text)
    
    return " ".join(tw)

In [10]:
data.tweet = data.tweet.apply(lambda x: tweet_cleaner(x))

### word2vec

In [11]:
documents = [text.split() for text in data.tweet]

In [12]:
len(documents)

7589

In [13]:
w2v_model = gensim.models.word2vec.Word2Vec(size = 256, window = 7, min_count = 5)

In [14]:
w2v_model.build_vocab(documents)

In [15]:
w2v_model.train(documents, total_examples=len(documents), epochs=32)

(1997973, 2751136)

In [37]:
w2v_model.wv["candy"]

array([ 0.24965465,  0.43766186,  0.19290075,  0.250684  ,  0.06314775,
       -0.01648778, -0.39419493, -0.36667046, -0.38766044,  0.31361103,
        0.20026943,  0.04479902, -0.14457338,  0.35593495, -0.22319797,
       -0.27218124,  0.3576334 ,  0.29744104, -0.01993314, -0.07666172,
        0.0562663 , -0.04541628, -0.29435062,  0.22949773,  0.13758448,
       -0.20978875, -0.04812137,  0.10120973,  0.12156332,  0.15375426,
       -0.1694644 ,  0.2937742 , -0.13212268,  0.19876514, -0.17425032,
       -0.13143541,  0.1331478 ,  0.09055082, -0.18663642, -0.1771004 ,
       -0.21306203,  0.24455754, -0.1206401 ,  0.21580699, -0.27825752,
       -0.03186091,  0.08516176, -0.32230636,  0.14660826, -0.0928966 ,
        0.01597192, -0.30821905, -0.05036233, -0.28636187, -0.15881534,
       -0.24499513,  0.13279466, -0.14328736,  0.08959588,  0.1776559 ,
       -0.20604728,  0.27289712,  0.06550793,  0.5347871 , -0.2856566 ,
        0.08157673, -0.15944466,  0.0705123 ,  0.02696706, -0.04

### Converting tweets to vectors

In [38]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.tweet)

In [39]:
len(tokenizer.word_index)

14849

In [40]:
x_train = pad_sequences(tokenizer.texts_to_sequences(data.tweet), maxlen=256, padding="post", truncating="post")

In [41]:
x_train

array([[ 2448,    95,   259, ...,     0,     0,     0],
       [ 6518,  2183,   143, ...,     0,     0,     0],
       [ 1003,  6520,     2, ...,     0,     0,     0],
       ...,
       [  822,   181,  2377, ...,     0,     0,     0],
       [   54,    15,  6516, ...,     0,     0,     0],
       [14847,   432,   555, ...,     0,     0,     0]], dtype=int32)

In [42]:
y_train = data.label

y_train_f = []
for x in y_train:
    if x == 1:
        y_train_f.append(1)
    elif x == 0:
        y_train_f.append(0)
    elif x == -1:
        y_train_f.append(2)
        
y_train_f = to_categorical(y_train_f)

In [43]:
y_train_f

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

### Model

In [44]:
embedding_matrix = np.zeros((14850,256))

In [45]:
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

In [46]:
embedding_layer = Embedding(14850, 256, weights=[embedding_matrix], input_length=256, trainable=False)

In [47]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.25))
model.add(Dense(200, activation="relu"))
model.add(Dropout(0.25))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(50, activation="relu"))
model.add(Dense(3, activation="softmax"))

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 256, 256)          3801600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256, 256)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 256, 200)          51400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256, 200)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 153       
Total para

In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [50]:
model.fit(x_train, y_train_f, batch_size=32, epochs=4, validation_split=0.1, verbose=1)

Train on 6830 samples, validate on 759 samples
Epoch 1/4
6830/6830 [==============================] - 92s 14ms/step - loss: 1.0010 - acc: 0.4902 - val_loss: 1.0313 - val_acc: 0.3939
Epoch 2/4
6830/6830 [==============================] - 112s 16ms/step - loss: 0.9945 - acc: 0.4966 - val_loss: 1.0158 - val_acc: 0.3939
Epoch 3/4
6830/6830 [==============================] - 96s 14ms/step - loss: 0.9947 - acc: 0.4966 - val_loss: 1.0310 - val_acc: 0.3939
Epoch 4/4
6830/6830 [==============================] - 123s 18ms/step - loss: 0.9945 - acc: 0.4966 - val_loss: 1.0183 - val_acc: 0.3939


### Testing

In [51]:
def sentiment(text):
    
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=256)
    score = model.predict([x_test])[0]
    
    final = "Positive = %f ,Negative = %f, Neutral = %f" % (score[1], score[2], score[0])
    return print(final)

In [52]:
sentiment("My teachers call themselves givng us candy")

Positive = 0.353107 ,Negative = 0.104911, Neutral = 0.541982
